In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install tensorflow==2.9.1 keras==2.9.0

import os
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Mount Google Drive to access files if needed
from google.colab import drive

drive.mount('/content/drive')

# Load the model
model = load_model("/content/drive/MyDrive/keras_model.h5", compile=False)

# Load the labels
with open("/content/drive/MyDrive/labels.txt", "r") as file:
    class_names = file.readlines()

# Create the array of the right shape to feed into the keras model
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("/content/drive/MyDrive/rain1.jpg").convert("RGB")

# # Replace this with the path to your image
# image = Image.open("/content/drive/MyDrive/rain1.jpg").convert("RGB")

# Resize the image to be at least 224x224 and then crop from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# Turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predict using the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name.strip())
print("Confidence Score:", confidence_score)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1/1 [==============================] - 1s 923ms/step
Class: 1 Rain
Confidence Score: 0.9436126


In [12]:
import os
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np
from sklearn.metrics import accuracy_score

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Mount Google Drive to access files if needed
from google.colab import drive

drive.mount('/content/drive')

# Load the model
model = load_model("/content/drive/MyDrive/keras_model.h5", compile=False)

# Load the labels
with open("/content/drive/MyDrive/labels.txt", "r") as file:
    class_names = file.readlines()
    print(class_names)
    category_to_index = {}
    for class_name in class_names:
        index, name = class_name.split(' ')
        name = name.strip()  # Removing any trailing whitespace or newline
        category_to_index[name] = int(index)  # Ensure the index is an integer

# Create the array of the right shape to feed into the keras model
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Save all the predictions and actual labels
predictions = []
labels = []

root_dir = "/content/drive/MyDrive/Image_validation"

# Process each image in the directory
for image_name in os.listdir(root_dir):
    image_path = os.path.join(root_dir, image_name)

    # Check if the current path is indeed a file and ends with an image extension
    if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        try:
            # Open the image and convert it to RGB if necessary
            image = Image.open(image_path).convert("RGB")

            # Resize the image to be 224x224
            size = (224, 224)
            image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

            # Turn the image into a numpy array
            image_array = np.asarray(image)

            # Normalize the image
            normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

            # Load the image into the array
            data[0] = normalized_image_array

            # Predict using the model
            prediction = model.predict(data)
            index = np.argmax(prediction)  # Get the index of the class with the highest probability
            class_name = class_names[index].strip()  # Remove any extra spaces or newline characters
            confidence_score = prediction[0][index]

            # Append the prediction and the actual label
            predictions.append(index)
            label_name = image_name.split('_')[0]  # Assuming the label is part of the filename (e.g., 'cloudy1.jpg' -> 'cloudy')
            labels.append(category_to_index.get(label_name, -1))  # Use -1 if label is not found

            # Print prediction and confidence score
            print(f"Image: {image_name}")
            print("Predicted Class:", class_name)
            print("Confidence Score:", confidence_score)

        except Exception as e:
            print(f"Error processing {image_name}: {e}")

# Optionally, compute accuracy or other metrics
if len(predictions) == len(labels):
    accuracy = accuracy_score(labels, predictions)
    print(f"Accuracy: {accuracy * 100:.2f}%")
else:
    print("Mismatch in number of predictions and labels")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['0 Cloudy\n', '1 Rain\n', '2 Shine\n', '3 Sunrise\n']
1/1 [==============================] - 2s 2s/step
Image: cloudy1.jpg
Predicted Class: 3 Sunrise
Confidence Score: 0.67571074
1/1 [==============================] - 0s 72ms/step
Image: cloudy2.jpg
Predicted Class: 0 Cloudy
Confidence Score: 0.8996364
1/1 [==============================] - 0s 69ms/step
Image: rain1.jpg
Predicted Class: 1 Rain
Confidence Score: 0.9436126
1/1 [==============================] - 0s 46ms/step
Image: sunrise1.jpg
Predicted Class: 2 Shine
Confidence Score: 0.9317908
Accuracy: 0.00%
